# Examples for QuantumAlgebra.jl

In [2]:
using QuantumAlgebra

In [3]:
# convience function to show both the original and normal_form version of an expression
dispnormal(x) = display("text/latex","\$ $(latex(x)) \\quad\\to\\quad $(latex(normal_form(x))) \$");

## Parameters
Create parameters with `param(:name,state,indices...)`, where `state = 'r','n','c'` stands for real, non-conjugated, or conjugated parameters. For more convenience, we also have the string macros `Pr"name"` for real parameters and `Pc"name"` for complex (non-conjugated) parameters. These macros support indices with `Pr"name_i,j"`.

In [4]:
ωij = param(:ω,'r',:i,:j)
α = param(:α,'n')
display(ωij)
display(α)

ω(ij) 

α 

In [5]:
display(ωij == Pr"ω_i,j")
display(ωij')
display(α')

true

ω(ij) 

α* 

`normal_form` also reorders parameters

In [6]:
x = α*α*ωij*α*ωij*α'
display(x)
normal_form(x)

α² ω(ij) α ω(ij) α* 

α* α³ ω(ij)² 

Parameters where state is not set are complex by default.

In [7]:
x = param(:g,2)*param(:g,1)*param(:g,'c',3)*param(:b)*param(:a)*param(:a)*param(:d,'c',3)*param(:f,'r',1)
dispnormal(x)
dispnormal(x')

## Operators
Three types of operators are currently supported: Bosonic, fermionic, and two-level system operators. By default, `QuantumAlgebra` defines `a` and `adag` as bosonic operators, `f` and `fdag` as fermionic operators, and `σx`,`σy`,`σz`,`σp`,`σm` as two-level system operators (`σp`/`σm` are $\sigma^\pm$).

In [8]:
dispnormal(a()*adag())

with indices

In [9]:
dispnormal(a(:i)*adag(:j))

Indices can be symbolic or integer:

In [10]:
dispnormal(a(:i)*adag(1))

Fermions anticommute

In [11]:
dispnormal(f(:i)*fdag(:j))

In [12]:
dispnormal(f(:i,:j)*fdag(:i,:l))

$f^2$ and $f^{\dagger 2}$ are normalized to $0$. Here, we also use that `f()' = fdag()`

In [13]:
dispnormal(f()*f())
dispnormal(f()'*f()')

Since normal form also gives canonical ordering of operators by indices, anticommutation of fermionic operators can introduce a minus sign.

In [14]:
dispnormal(f(:j)*f(:i))

Indices in $\delta$s are automatically ordered and simplified (so $\delta_{ik}\delta_{kj} \to \delta_{ij}\delta_{ik}$).

In [15]:
dispnormal(f(:i,:k)*fdag(:k,:j))

Bosonic and fermionic operators commute (independent of indices).

In [16]:
dispnormal(a(:i)*f(:i)*fdag(:j))

Define a second bosonic species with name `b`. These always commute with other species.

In [17]:
@boson_ops b;

In [18]:
dispnormal(a(:i)*b(:i)*bdag(:j))

By default, operators for two-level systems (Pauli matrices) are written in terms of the Hermitian matrices $\sigma^x,\sigma^y,\sigma^z$ (we use superscripts so as to avoid possible confusion with indices).

In [19]:
display(σx(3))
display(σz(3))
display(σp(1))

σˣ(3)

σᶻ(3)

1//2 σˣ(1) + 1//2i σʸ(1)

`normal_form` contracts products of Pauli matrices

In [20]:
dispnormal(σx()*σx())

... but not if indices are different (since the expression would be $\delta_{ij} + \sigma^x_i \sigma^x_j (1-\delta_{ij})$)

In [21]:
dispnormal(σx(:i)*σx(:j))

Commutation is performed, using $[\sigma^a, \sigma^b] = 2 i \varepsilon_{a b c} \sigma^c$ 

In [22]:
dispnormal(σz(:i)*σx(:j))

Excitation/deexcitation operators $\sigma^\pm$ get rewritten in terms of $\sigma^{x,y,z}$ by default.

In [23]:
dispnormal(σm(:j)*σp(:i))

With `QuantumAlgebra.use_σpm()`, we can ask to use the excitation/deexcitation operators $\sigma^+$ and $\sigma^-$ instead of the Pauli matrices $\sigma^{x,y,z}$ as the "natural" basis.

Note that exchanging them easily produces "complicated" expressions since they are neither bosons nor fermions, but most properly thought of as "hard-core bosons".

In [24]:
QuantumAlgebra.use_σpm()
dispnormal(σm(:j)*σp(:i))

Use `QuantumAlgebra.use_σxyz()` to switch back to $\sigma^{x,y,z}$.

In [25]:
QuantumAlgebra.use_σxyz()
dispnormal(σm(:j)*σp(:i))

Scalars are incorporated "naturally", with the code trying to use integers and rationals where possible.

In [26]:
dispnormal(a()*adag() - 1)
dispnormal(1//2*a()*adag()*adag())
dispnormal(5//4*σx()*σy()*σz())
dispnormal(6//3*σx()*σy()*σz())
dispnormal(σp(1) * σm(1) - 1//2)

## Analytic sums
Define sums over indices with `∑(ind`, where `∑` is entered as `\sum<tab>` (note that this is **not** `Σ = \Sigma<tab>`). Sum indices are automatically renamed to the special symbol `#ᵢ` (with $i=1,2,\ldots$) and reordered since they have no definite identity.
Sums over $\delta$s disappear automatically.

In [27]:
dispnormal(∑(:i, a(:i)*adag(:j)))
dispnormal(∑(:i, Pr"g_i"*a(:i)*adag(:j)))

In [28]:
H = ∑(:i,Pr"ω_i"*adag(:i,:i))
display(H)
dispnormal(a(:k,:k)*H)

∑₁ ω(#₁) a†(#₁#₁)

In [29]:
H = ∑(:i,Pr"ω_i"*adag(:i)*a(:i))
display(H)
dispnormal(a(:k)*H)

∑₁ ω(#₁) a†(#₁) a(#₁)

In [30]:
x = ∑(:i,adag(:i)*a(:i))
dispnormal(x)
dispnormal(adag(:n)*x)
dispnormal(a(:n)*x)
dispnormal(σz(:n)*x)
dispnormal(Pc"g_n"*a(:n)*x)
dispnormal(x*adag(:n))
dispnormal(x*adag(:n)*adag(:n))
dispnormal(x*adag(:n)*a(:n))

## Commutators

In [31]:
dispnormal(comm(σx(5),σy(3)))
dispnormal(comm(σx(5),σx(5)))
dispnormal(comm(σx(1),σz(1)))
dispnormal(2*comm(σx(5),σy(5)))
dispnormal(comm(σy(5),comm(σx(5),σy(5))))

In [32]:
dispnormal(comm(2//5*param(:h)*σx(5),3*param(:g)*σy(5)))

In [33]:
dispnormal(comm(σp(1),σp(1)))
dispnormal(comm(σp(1),σm(1)))
dispnormal(comm(σm(1),σp(1)))
dispnormal(comm(σm(1),σm(1)))
dispnormal(comm(σm(1),σp(2)))

In [34]:
dispnormal(comm(σm(1),σz(1)))

In [35]:
dispnormal(comm(a(2),adag(1)))

In [36]:
dispnormal(comm(a(1),adag(1)*a(1)))
dispnormal(comm(a(1),adag(1)*a(2)*a(1)))
dispnormal(comm(a(1),a(2)*adag(1)*a(1)*adag(2)))

In [37]:
dispnormal(σp(1)*σz(1) + σp(1))

## Many-mode Tavis-Cummings

In [38]:
H = ∑(:i,Pr"ω_i"*adag(:i)*a(:i)) + ∑(:j,1//2*Pr"ωe_j"*σz(:j)) + ∑(:i,∑(:j,Pr"g_i,j"*(adag(:i)+a(:i))*σx(:j)))

1//2 ∑₁ ωe(#₁) σᶻ(#₁) + ∑₁₂ g(#₁#₂) a†(#₁) σˣ(#₂) + ∑₁ ω(#₁) a†(#₁) a(#₁) + ∑₁₂ g(#₁#₂) a(#₁) σˣ(#₂)

In [39]:
tmp1 = normal_form(comm(adag(:n)*a(:m),H))
tmp2 = normal_form(comm(adag(:n)*a(:n),H))
display(tmp1)
display(tmp2)
display(tmp2 - QuantumAlgebra.replace_inds(QuantumAlgebra.OpIndex(:m)=>QuantumAlgebra.OpIndex(:n))(tmp1))

∑₁ g(m#₁) a†(n) σˣ(#₁) + ω(m) a†(n) a(m) - ω(n) a†(n) a(m) - ∑₁ g(n#₁) σˣ(#₁) a(m)

∑₁ g(n#₁) a†(n) σˣ(#₁) - ∑₁ g(n#₁) σˣ(#₁) a(n)

0

In [40]:
for op in [a(:n),adag(:n),σx(:k),σy(:k),σz(:k)]
    display("text/latex",string("\$[",latex(op),",H] = ",latex(normal_form(comm(op,H))),"\$"))
end

In [41]:
for op in [adag(:n)*σz(:k),a(:n)*σz(:k)]
    display("text/latex",string("\$[",latex(op),",H] = ",latex(normal_form(comm(op,H))),"\$"))
end

## Cumulant expansions
`expval_as_corrs` expresses expectation values of operator products in terms of correlators.

In [42]:
expval(H)

1//2 ∑₁ ωe(#₁) ⟨σᶻ(#₁)⟩  + ∑₁₂ g(#₁#₂) ⟨a†(#₁) σˣ(#₂)⟩  + ∑₁ ω(#₁) ⟨a†(#₁) a(#₁)⟩  + ∑₁₂ g(#₁#₂) ⟨a(#₁) σˣ(#₂)⟩ 

In [43]:
expval_as_corrs(H)

1//2 ∑₁ ωe(#₁) ⟨σᶻ(#₁)⟩c  + ∑₁₂ g(#₁#₂) ⟨a†(#₁)⟩c ⟨σˣ(#₂)⟩c  + ∑₁ ω(#₁) ⟨a†(#₁)⟩c ⟨a(#₁)⟩c  + ∑₁₂ g(#₁#₂) ⟨σˣ(#₂)⟩c ⟨a(#₁)⟩c  + ∑₁₂ g(#₁#₂) ⟨a†(#₁) σˣ(#₂)⟩c  + ∑₁ ω(#₁) ⟨a†(#₁) a(#₁)⟩c  + ∑₁₂ g(#₁#₂) ⟨a(#₁) σˣ(#₂)⟩c 

In [44]:
display(expval_as_corrs(a(2)))
display(expval_as_corrs(3*a(2)))

⟨a(2)⟩c 

3 ⟨a(2)⟩c 

In [45]:
display(expval_as_corrs(a(2)*a(2)))
display(expval_as_corrs(2*a(1)*a(2)*a(3)))
display(expval_as_corrs(a(1)*a(2)*a(3)*a(4)))
display(expval_as_corrs(adag(2)*a(1)*σz(1)))

⟨a(2)⟩c²  + ⟨a(2)²⟩c 

2 ⟨a(1)⟩c ⟨a(2)⟩c ⟨a(3)⟩c  + 2 ⟨a(1)⟩c ⟨a(2) a(3)⟩c  + 2 ⟨a(2)⟩c ⟨a(1) a(3)⟩c  + 2 ⟨a(3)⟩c ⟨a(1) a(2)⟩c  + 2 ⟨a(1) a(2) a(3)⟩c 

⟨a(1)⟩c ⟨a(2)⟩c ⟨a(3)⟩c ⟨a(4)⟩c  + ⟨a(1)⟩c ⟨a(2)⟩c ⟨a(3) a(4)⟩c  + ⟨a(1)⟩c ⟨a(3)⟩c ⟨a(2) a(4)⟩c  + ⟨a(1)⟩c ⟨a(4)⟩c ⟨a(2) a(3)⟩c  + ⟨a(1)⟩c ⟨a(2) a(3) a(4)⟩c  + ⟨a(2)⟩c ⟨a(3)⟩c ⟨a(1) a(4)⟩c  + ⟨a(2)⟩c ⟨a(4)⟩c ⟨a(1) a(3)⟩c  + ⟨a(2)⟩c ⟨a(1) a(3) a(4)⟩c  + ⟨a(3)⟩c ⟨a(4)⟩c ⟨a(1) a(2)⟩c  + ⟨a(3)⟩c ⟨a(1) a(2) a(4)⟩c  + ⟨a(4)⟩c ⟨a(1) a(2) a(3)⟩c  + ⟨a(1) a(2)⟩c ⟨a(3) a(4)⟩c  + ⟨a(1) a(3)⟩c ⟨a(2) a(4)⟩c  + ⟨a(1) a(4)⟩c ⟨a(2) a(3)⟩c  + ⟨a(1) a(2) a(3) a(4)⟩c 

⟨a†(2)⟩c ⟨σᶻ(1)⟩c ⟨a(1)⟩c  + ⟨a†(2)⟩c ⟨a(1) σᶻ(1)⟩c  + ⟨σᶻ(1)⟩c ⟨a†(2) a(1)⟩c  + ⟨a(1)⟩c ⟨a†(2) σᶻ(1)⟩c  + ⟨a†(2) a(1) σᶻ(1)⟩c 

In [46]:
display(expval_as_corrs( Pr"g"*adag(3)*a(2)*a(2)))
display(expval_as_corrs(-Pr"g_1"*σz(1)))

g ⟨a†(3)⟩c ⟨a(2)⟩c²  + g ⟨a†(3)⟩c ⟨a(2)²⟩c  + 2 g ⟨a(2)⟩c ⟨a†(3) a(2)⟩c  + g ⟨a†(3) a(2)²⟩c 

-g(1) ⟨σᶻ(1)⟩c 

## Vacuum expectation values

In [47]:
display(vacA(Avac(σp(1)*σm(1))))
display(vacA(Avac(σm(1)*σp(1))))

0

1

In [48]:
# silly trick to get a "1" as a QuantumAlgebra object without using the internals
ident = 1 + 0*a()

1

In [49]:
nphotstate(n,ind) = 1/sqrt(factorial(n)) * *((adag(ind) for i=1:n)...)
for i2 = (:n,:m)
    stateop = 1/√2*nphotstate(3,:n) + 1/√2*nphotstate(1,i2)
    display("text/latex",string("\$|\\psi\\rangle = \\left(",latex(stateop),"\\right)|0\\rangle\$"))
    for A in [ident,adag(:n)*a(:n),adag(:n)*adag(:n)*a(:n)*a(:n)]
        display("text/latex",string("\$\\langle\\psi|",latex(A),"|\\psi\\rangle = ",latex(vacExpVal(A,stateop)),"\$"))
    end
    i2 == :n && display("text/latex","")
end

## Convert QuantumAlgebra expressions to julia expressions (code)

For converting to code, we cannot have bare operators, but only expectation values or correlators/cumulants. An expectation value $\langle a^\dagger_i a_j\rangle$ is represented as a two-dimensional array `aᴴa[i,j]`, etc. Sums are not written explicitly, but indicated by special sum index names `s̄ᵢ`, which are not possible "normal" index symbols (since `"s̄"` is a two-character unicode string, and we only allow single-character indices). $\delta$ are converted to `I` (code using this should have `using LinearAlgebra`, which defines `I` as the `UniformScaling` type, with `I[i,j] = δᵢⱼ`).

In [50]:
x = ∑(:i, a(:i)*adag(:j)) + f(:k_1)*fdag(:k_2)
x = expval(normal_form(x))
display(x)
julia_expression(x)

1 + δ(k₁k₂)  + ∑₁ ⟨a†(j) a(#₁)⟩  - ⟨f†(k₂) f(k₁)⟩ 

:(1.0 + I[k₁, k₂] + aᴴa[j, s̄₁] + -1.0 * fᴴf[k₂, k₁])